# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract image data 

## Read dataframe

In [5]:
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/',
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


interactive(children=(Select(description='x', options=('scratch', 'proj'), value='scratch'), Output()), _dom_c…

In [7]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20200529_111342_seed3273_80epochs', '20200626_075510…

In [41]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200911_084729_bsize64_spec_test_128_withspec


In [42]:
# main_dir=''

In [43]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

In [44]:
print(df.shape)
df.head()


(63288, 20)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
0,0,0,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,188.919404,4.752520,0.013036,193.684960,12.402488,26.132530,17735.385362,2.903150e+07,0,0,"[0.00021404027994051573, 0.05836540469577179, ...","[5.26696921090502e-05, 0.00262919138735563, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[56721116.89526159, 2266.702487637289, 1044.62...","[49429.16516472229, 68.58415225759842, 37.4961...",0-0
1,0,1,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,238.546729,5.310422,0.013186,243.870336,14.276295,30.869478,9143.077011,1.376490e+07,0,0,"[4.881620419695972e-05, 0.029940962837564058, ...","[1.1176921406708908e-05, 0.0016304849638566455...","[-1.031746031584782, -0.6161616169043975, -0.3...","[52195020.53115036, 5132.58483118657, 1822.340...","[95252.78262594297, 165.06217170762582, 58.703...",0-1
2,0,2,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,304.916736,6.836572,0.013186,311.766495,16.581349,35.575580,5711.176888,9.634616e+06,0,0,"[7.510185261070727e-06, 0.013934880516536756, ...","[3.635852805382416e-06, 0.0008010749185763909,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[47675984.306957416, 11910.89646540396, 3561.1...","[153839.8039633917, 289.3219609003277, 95.0698...",0-2
3,0,3,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,400.074667,9.812852,0.013186,409.900704,19.550103,39.014045,5015.937476,1.228355e+07,0,0,"[0.0, 0.005192756660830974, 0.1328552366441135...","[0.0, 0.00034168984835056563, 0.00249839092279...","[-1.031746031584782, -0.6161616169043975, -0.3...","[43227220.757471606, 20324.55559345614, 5752.5...","[174030.0172876506, 488.5074326366468, 143.205...",0-3
4,0,4,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,532.870231,15.785946,0.013186,548.669362,23.470681,42.622959,5196.576184,1.462953e+07,0,0,"[0.0, 0.0011624693888451162, 0.068439155875659...","[0.0, 8.522798294354985e-05, 0.001605879296650...","[-1.031746031584782, -0.6161616169043975, -0.3...","[38971620.06674719, 29259.28363438591, 8084.64...","[166732.25959719243, 574.4702355005246, 159.81...",0-4


### View best epochs

#### Locations with best chi_sqr

In [45]:
chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]

,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
1282,0,1282,10.706573,10820.378280,0.013186,10831.098039,141.660881,39.156213,1458.189800,2.629087e+09
0,0,0,188.919404,4.752520,0.013036,193.684960,12.402488,26.132530,17735.385362,2.903150e+07
477,0,477,13.127615,9492.792528,0.008438,9505.928582,126.981460,59.669286,1247.837919,7.900354e+08
0,0,0,188.919404,4.752520,0.013036,193.684960,12.402488,26.132530,17735.385362,2.903150e+07
0,0,0,188.919404,4.752520,0.013036,193.684960,12.402488,26.132530,17735.385362,2.903150e+07
63220,17,63220,16.156006,14688.477377,0.013186,14704.646569,209.207705,6.094591,1273.719346,2.178192e+11
49,0,49,248.125622,4868.829357,4.006352,5120.961332,96.944632,86.778018,807.383581,2.832183e+07
2,0,2,304.916736,6.836572,0.013186,311.766495,16.581349,35.575580,5711.176888,9.634616e+06


In [46]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

In [47]:
df_sliced=f_get_best_chisqr_models(df,cutoff=0.2)
print(df_sliced.shape)

(7389, 20)


In [48]:
df_sliced.sort_values(by=['chi_1','chi_spec1'])[['epoch','step','chi_1','chi_spec1']].head(5)

,epoch,step,chi_1,chi_spec1
34,0,34,4201.551260,1204.174463
36,0,36,4274.674007,1069.077403
35,0,35,4293.791557,1186.198969
39,0,39,4420.645934,948.578775
37,0,37,4451.528190,1070.019120


In [50]:
### Plot chi-sqr values
df_sliced.plot(x="step", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### High Pixel images

In [20]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

In [21]:
df[df.num_vlarge>10].tail()

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label


## Compare samples

In [22]:
# df_sliced.sort_values(by=['chi_spec1','chi_1']).head(5)
# df_sliced.sort_values(by=['chi_1','chi_spec1']).head(5)

In [23]:
best_idx=[]
best_idx.append(df_sliced.sort_values(by=['chi_1','chi_spec1']).head(2).index)
best_idx.append(df_sliced.sort_values(by=['chi_spec1','chi_1']).head(2).index)

best_idx=[i for j in best_idx for i in j]
print(best_idx)

df_best=df_sliced.loc[best_idx]
print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])
df_best

[34, 36, 49, 48]
[(0, 34), (0, 36), (0, 49), (0, 48)]


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
34,0,34,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,590.710709,3610.827515,0.013036,4201.551260,87.289336,89.035114,1204.174463,2.290114e+07,0,0,"[0.0, 0.0012981891652077437, 0.009380072363305...","[0.0, 4.580298952153858e-05, 4.992494563465199...","[-1.031746031584782, -0.6161616169043975, -0.3...","[9890686.966629982, 39625.24396729469, 15799.8...","[63438.63444918153, 965.8519893054748, 333.404...",0-34
36,0,36,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,549.822424,3724.838979,0.012604,4274.674007,87.733099,73.155404,1069.077403,2.996282e+07,0,0,"[0.0, 0.004053890711353272, 0.0058130025913444...","[0.0, 0.00014669119504197043, 0.00018837386203...","[-1.031746031584782, -0.6161616169043975, -0.3...","[9559358.837982178, 38131.056941986084, 16025....","[50151.40836476784, 871.2066856743883, 310.111...",0-36
49,0,49,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,248.125622,4868.829357,4.006352,5120.961332,96.944632,86.778018,807.383581,2.832183e+07,0,0,"[0.0007266104240085929, 0.01069235803343833, 0...","[2.3960564237787163e-05, 0.0001139263347386569...","[-1.031746031584782, -0.6161616169043975, -0.3...","[8006246.397144318, 48566.62690639496, 23071.7...","[41429.362450126595, 1000.2392182530676, 354.1...",0-49
48,0,48,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,275.473472,4731.392390,3.378159,5010.244021,95.637351,88.348677,809.172239,2.803861e+07,0,0,"[0.0005576312556345015, 0.009217143072974983, ...","[2.7384795039786973e-05, 0.0001008231452980624...","[-1.031746031584782, -0.6161616169043975, -0.3...","[8131697.471200943, 51425.050594329834, 24058....","[41637.825331117674, 1019.4673728655655, 392.6...",0-48


## Plot pixel intensity and spectrum

In [24]:
def f_plot_hist_spec_combined(df,dict_bkg):

    fig=plt.figure(figsize=(6,6))
    ax1=fig.add_subplot(121)
    ax2=fig.add_subplot(122)
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):

        x1=row.hist_bin_centers
        y1=row.hist_val
        yerr1=row.hist_err
        x1=f_invtransform(x1)

        y2=row.spec_val
        yerr2=row.spec_err
        x2=np.arange(len(y2))

        label='{0}_{1}_{2}'.format(i,row.epoch,row.step)
        ax1.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
    #     ax2.errorbar(x2,y2,yerr2,marker=marker,markersize=5,linestyle='',label='{0}-{1}'.format(epoch,step))

        ax2.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
        ax2.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
    x=f_invtransform(x)
    ax1.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   

    y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
    x=np.arange(len(y))
    ax2.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)

    plt.legend()
    # plt.yscale('log')
    ax1.set_xscale('symlog',linthreshx=50)
    ax1.set_yscale('log')
    ax2.set_yscale('log')
    
    


In [25]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[:4000][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)


(4000, 128, 128)


In [31]:
f_plot_hist_spec_combined(df_best,dict_val)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## View image block

In [36]:
df[df.epoch>10]

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err,label
38676,11,38676,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,17.413503,14718.020052,0.013186,14735.446740,208.587528,8.736784,1287.268072,1.858436e+11,0,0,"[0.023317247689309355, 0.013642787954365018, 0...","[1.5580170184932295e-05, 3.150197114286887e-05...","[-1.031746031584782, -0.6161616169043975, -0.3...","[5421885.069244385, 69941.56492614746, 47984.6...","[450.3870089866078, 18.939607291930606, 11.777...",11-38676
38677,11,38677,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,18.450719,14183.082935,0.013186,14201.546840,202.707620,7.802094,1240.984393,2.948564e+11,0,0,"[0.02333977824509257, 0.0138611197685136, 0.01...","[1.2381044232205593e-05, 2.75888863141721e-05,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[5519532.034667969, 69779.6819152832, 47995.90...","[357.3801552914606, 20.2003517342118, 10.45905...",11-38677
38678,11,38678,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,16.714732,15204.274147,0.013186,15221.002065,213.953491,8.671777,1337.015811,1.652170e+11,0,0,"[0.0232045949103933, 0.013400852700849024, 0.0...","[1.3039741641160348e-05, 3.311027879240587e-05...","[-1.031746031584782, -0.6161616169043975, -0.3...","[5322867.395294189, 68612.21385192871, 47598.7...","[477.9318830855148, 21.286902379938535, 12.292...",11-38678
38679,11,38679,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,16.831644,15108.897877,0.013186,15125.742708,212.993350,8.795928,1327.018580,1.723991e+11,0,0,"[0.023215860188284903, 0.013459861299267559, 0...","[1.0680478778283034e-05, 2.4139042966347504e-0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[5342459.376708984, 68765.54179382324, 47706.9...","[467.81043290637285, 19.95164911471652, 11.412...",11-38679
38680,11,38680,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,19.228539,13809.900349,0.013186,13829.142074,198.613108,8.194102,1207.162159,2.211640e+11,0,0,"[0.023484349311368162, 0.013754904291360236, 0...","[1.738632218821384e-05, 3.446292859367856e-05,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[5594446.83404541, 70379.9592590332, 48488.234...","[412.487230135511, 18.945631148912483, 11.3955...",11-38680
38681,11,38681,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,19.123473,13963.073673,0.013186,13982.210332,200.235349,8.615394,1214.939206,1.239787e+11,0,0,"[0.023572593988185737, 0.01408830287242496, 0....","[1.2886777501822884e-05, 3.35408702874789e-05,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[5576738.42767334, 71618.37571716309, 49135.00...","[550.9903997226455, 20.697193606723083, 12.339...",11-38681
38682,11,38682,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,15.958347,15410.565906,0.013186,15426.537438,216.148685,8.613018,1347.184529,1.581381e+11,0,0,"[0.023375451625082654, 0.013453960439425705, 0...","[1.4291260471567677e-05, 1.9120971266349872e-0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[5303063.774597168, 70407.86162567139, 48641.8...","[488.5458349140024, 22.956069277670295, 12.286...",11-38682
38683,11,38683,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,15.611117,15773.015557,0.013186,15788.639860,220.143550,8.697858,1390.431629,2.065304e+11,0,0,"[0.023198962271447493, 0.013359546681956049, 0...","[1.5224342216098262e-05, 2.4685067759294532e-0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[5222173.51171875, 68833.34466552734, 47884.78...","[427.67204735012854, 19.841077339457406, 11.56...",11-38683
38684,11,38684,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,16.840809,14884.702979,0.013186,14901.556974,210.593032,9.025395,1305.863080,1.392760e+11,0,0,"[0.023274064124058205, 0.0134864151685559, 0.0...","[1.6341125957920558e-05, 2.632494126291401e-05...","[-1.031746031584782, -0.6161616169043975, -0.3...",

In [37]:
def f_get_img(df,epoch,step):
    df_temp=df[(df.epoch==epoch)&(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,11,38676)

In [38]:
f_plot_grid(img_arr[:18],cols=6,fig_size=(10,6))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,6))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
df_sliced[df_sliced.epoch==4]['step'].values

In [ ]:
df[(df.epoch==18)&(df.step>=15975)&(df.step<=15983)][['epoch','step','chi_1','chi_spec1','num_vlarge']]